## Análisis de texto Orfeo con LLM open source

El proceso se organiza en cinco etapas secuenciales:

1. Parámetros
2. Autenticación Onedrive
3. Extracción del texto desde pdf
4. Análisis de texto con LLM: Llama y Mistral
5. Consolidar y guardar los resultados

## 1. Parámetros

In [ ]:
import os

## 1. INPUTS
link = "https://planeacionnacional-my.sharepoint.com/:f:/r/personal/johasuarez_dnp_gov_co/Documents/Consolidado%20Orfeo%20Comunicaciones?csf=1&web=1&e=f37mjQ"
carpeta_procesar='2025/ENTRADAS'


## 2. MODELOS
INSTRUCCION = (
    "Lee el TEXTO y responde en tres items numerados:\n"
    "1) Tema principal.\n",
    "2) Cuál es el origen de la solicitud.\n"
    "3) ¿Qué tipo de solicitud es?\n"
    "4) Dame un resumen en 7 palabras.")

MODELOS = [
    {"name": "llama",   "path": r"C:\Users\Sebastian\Documents\DNP\Orfeo\modelos\meta-llama-3-8b-instruct.Q4_K_M.gguf"},
    {"name": "mistral", "path": r"C:\Users\Sebastian\Documents\DNP\Orfeo\modelos\mistral-7b-instruct-v0.1.Q4_K_M.gguf"},]

## 3. OUTPUTS
EXCEL_CONSOLIDADO = "Orfeo_analisis_llm.xlsx"
EXCEL_LLAMA = "Orfeo_llama.xlsx"
EXCEL_MISTRAL = "Orfeo_mistral.xlsx"

## 2. Autenticación Onedrive

In [ ]:
import base64
import msal
import requests

# ==== 1. Datos base ====
CLIENT_ID = "d3590ed6-52b3-4102-aeff-aad2292ab01c"  # App pública de Microsoft
AUTHORITY = "https://login.microsoftonline.com/common"
SCOPE = ["Files.Read.All"]

# ==== 2. Iniciar flujo interactivo con MFA ====
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
print(flow["message"])  # -> Abre la URL que muestra y pega el código

result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception("Error al autenticar:", result.get("error_description"))

token = result["access_token"]
print("Autenticación exitosa ✅")

# ==== 3. Preparar el enlace compartido ====


# Codificar el link para Graph API
encoded_link = base64.urlsafe_b64encode(link.encode()).decode().rstrip("=")

# Endpoint para listar archivos
url = f"https://graph.microsoft.com/v1.0/shares/u!{encoded_link}/driveItem/children"

headers = {"Authorization": f"Bearer {token}"}

metadata_url = f"https://graph.microsoft.com/v1.0/shares/u!{encoded_link}/driveItem"
meta_resp = requests.get(metadata_url, headers=headers).json()

drive_id = meta_resp["parentReference"]["driveId"]
item_id = meta_resp["id"]

# =========================
# FUNCIÓN PARA LISTAR LOS ARCHIVOS
# =========================
def listar_hijos(drive_id, item_id):
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/children"
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print("Error al listar hijos:", resp.text)
        return []
    return resp.json().get("value", [])

# =========================
#FILTRAR LOS ARCHIVOS DE EXCEL (HOJAS DE VIDA)
# =========================
def recorrer_recursivo(drive_id, item_id, ruta_base=""):
    archivos = []
    items = listar_hijos(drive_id, item_id)

    for item in items:
        nombre = item["name"]
        nueva_ruta = f"{ruta_base}/{nombre}" if ruta_base else nombre

        if "folder" in item:
            # Recursión para subcarpetas
            archivos.extend(recorrer_recursivo(drive_id, item["id"], nueva_ruta))
        else:
            # Filtrar solo archivos .xlsx
            if nombre.lower().endswith(".pdf"):
                archivos.append({
                    "nombre": nombre,
                    "ruta": nueva_ruta,
                    "id": item["id"]
                })
    return archivos

# =========================
# LISTA CON TODOS LOS ARCHIVOS EN LA CARPETA
# =========================
todos_los_archivos = recorrer_recursivo(drive_id, item_id)

# Función para obtener archivos de una carpeta específica
def filtrar_por_carpeta(lista, carpeta_objetivo):
    return [
        archivo for archivo in lista
        if archivo["ruta"].startswith(carpeta_objetivo)]

# Filtrar archivos de cada carpeta
archivos_procesar = filtrar_por_carpeta(todos_los_archivos, carpeta_procesar)

# Convertir a sets de nombres para comparar
nombres_procesar = {a["nombre"] for a in archivos_procesar}

#print(f"Total archivos .xlsx: {len(todos_los_archivos)}")
#Archivo para procesar
print(f"Archivos pdf: {len(nombres_procesar)}")

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FQ2PE5RNZ to authenticate.
Autenticación exitosa ✅


[{'nombre': '120253100125753.pdf',
  'ruta': '2025/ENTRADAS/120253100125753.pdf',
  'id': '01WW347OZV7IJRY5L62VCLNZ5OAJXVMS5C'},
 {'nombre': '120253240150443.pdf',
  'ruta': '2025/ENTRADAS/120253240150443.pdf',
  'id': '01WW347OY5SYKMPX6FQBF3C3ASBO5QSEYE'},
 {'nombre': '120253240164933.pdf',
  'ruta': '2025/ENTRADAS/120253240164933.pdf',
  'id': '01WW347O7ZOO2JCUQ5KFA2F6QN242XY3V2'},
 {'nombre': '120253240165013.pdf',
  'ruta': '2025/ENTRADAS/120253240165013.pdf',
  'id': '01WW347O6BK4HZEFJFNNEZUFQNNK7FVAMM'},
 {'nombre': '120253400090033.pdf',
  'ruta': '2025/ENTRADAS/120253400090033.pdf',
  'id': '01WW347O5JPRD5WDQUHZF2N7PP3IJMGKLZ'},
 {'nombre': '120253400122063.pdf',
  'ruta': '2025/ENTRADAS/120253400122063.pdf',
  'id': '01WW347OZ4VN5AUOS7KBFZOVUZECRJEYPE'},
 {'nombre': '120253400184473.pdf',
  'ruta': '2025/ENTRADAS/120253400184473.pdf',
  'id': '01WW347OZR4LOBAAVICNC3KM2B46H4IWNP'},
 {'nombre': '120254240002993.pdf',
  'ruta': '2025/ENTRADAS/120254240002993.pdf',
  'id': '01WW34

## 3. Extraccion del texto desde pdf

In [ ]:
import pandas as pd
from io import BytesIO
from pdfminer.high_level import extract_text

# ========================================
# FUNCIÓN PARA DESCARGAR Y EXTRAER TEXTO
# ========================================
def extraer_texto_pdf(item, drive_id, headers):
    """
    item: diccionario con claves {'nombre','ruta','id'}
    drive_id: id de la unidad en OneDrive/SharePoint
    headers: encabezados con el token de acceso
    """
    try:
        # Descargar contenido del PDF
        url_pdf = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item['id']}/content"
        resp_pdf = requests.get(url_pdf, headers=headers)

        if resp_pdf.status_code != 200:
            print(f"❌ Error al descargar {item['nombre']}: {resp_pdf.status_code}")
            return ""

        pdf_bytes = resp_pdf.content

        # Extraer texto sin OCR
        with BytesIO(pdf_bytes) as f:
            texto = extract_text(f)

        return texto
    
    except Exception as e:
        print(f"❌ Error procesando {item['nombre']}: {e}")
        return ""


# ========================================
# RECORRER TODOS LOS ARCHIVOS Y EXTRAER TEXTO
# ========================================
registros = []

for item in archivos_procesar:
    print(f"📄 Procesando: {item['nombre']} ...")

    texto = extraer_texto_pdf(item, drive_id, headers)

    registros.append({
        "nombre": item["nombre"],
        "ruta": item["ruta"],
        "id": item["id"],
        "texto": texto
    })

# ========================================
# Convertir a DataFrame
# ========================================
df_pdfs = pd.DataFrame(registros)
df_pdfs.head()


📄 Procesando: 120253100125753.pdf ...
📄 Procesando: 120253240150443.pdf ...
📄 Procesando: 120253240164933.pdf ...


,nombre,ruta,id,texto
0,120253100125753.pdf,2025/ENTRADAS/120253100125753.pdf,01WW347OZV7IJRY5L62VCLNZ5OAJXVMS5C,
1,120253240150443.pdf,2025/ENTRADAS/120253240150443.pdf,01WW347OY5SYKMPX6FQBF3C3ASBO5QSEYE,"MEMORANDO \n\nBogotá D.C., lunes, 07 de julio ..."
2,120253240164933.pdf,2025/ENTRADAS/120253240164933.pdf,01WW347O7ZOO2JCUQ5KFA2F6QN242XY3V2,JUZGADO PROMISCUO MUNICIPAL DE \nCOTORRA-CÓRD...


## 4. Análisis de texto con LLM: Llama y Mistral


In [6]:
# ==========================================
# ✅ df_pdfs -> 2 TXT por texto en OneDrive
#    Carpeta: orfeo_salida_llm
# ==========================================

import os, time, gc, platform, json
import pandas as pd
from llama_cpp import Llama
import requests

# Si True, no vuelve a crear si el TXT ya existe en OneDrive
SKIP_IF_EXISTS = True

PARAMS = dict(
    n_ctx=2048,
    max_tokens=160,
    temperature=0.2,
    top_p=0.95,
    repeat_penalty=1.1,
)

# Config GPU/CPU
gpu_ok = os.system("nvidia-smi >NUL 2>&1") == 0 if platform.system().lower().startswith("win") \
         else os.system("nvidia-smi >/dev/null 2>&1") == 0
N_GPU_LAYERS = -1 if gpu_ok else 0
N_BATCH = 512 if gpu_ok else 128

try:
    import multiprocessing as mp
    N_THREADS = max(2, mp.cpu_count() // 2)
except Exception:
    N_THREADS = 4

# -------------------------------
# (0) df_pdfs en memoria
# -------------------------------
# Se asume que df_pdfs ya existe y tiene columnas: nombre, ruta, id, texto
assert {"nombre", "ruta", "id", "texto"}.issubset(df_pdfs.columns)

# -------------------------------
# (1) Helpers OneDrive / Graph
# -------------------------------

def obtener_o_crear_carpeta_salida(nombre_carpeta, drive_id, parent_id_root, headers):
    """
    Busca una carpeta dentro de parent_id_root.
    Si no existe, la crea. Devuelve el item_id de esa carpeta.
    """
    # 1) Buscar si ya existe
    url_list = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{parent_id_root}/children"
    resp = requests.get(url_list, headers=headers)
    if resp.status_code != 200:
        raise Exception(f"Error listando hijos de la raíz: {resp.status_code} - {resp.text}")
    data = resp.json().get("value", [])

    for it in data:
        if it.get("name") == nombre_carpeta and "folder" in it:
            print(f"📁 Carpeta '{nombre_carpeta}' ya existe en OneDrive.")
            return it["id"]

    # 2) Crear carpeta si no existe
    print(f"📁 Creando carpeta '{nombre_carpeta}' en OneDrive...")
    url_create = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{parent_id_root}/children"
    payload = {
        "name": nombre_carpeta,
        "folder": {},
        "@microsoft.graph.conflictBehavior": "rename"
    }
    resp_create = requests.post(url_create, headers={**headers, "Content-Type": "application/json"},
                                data=json.dumps(payload))
    if resp_create.status_code not in (200, 201):
        raise Exception(f"Error creando carpeta: {resp_create.status_code} - {resp_create.text}")

    carpeta = resp_create.json()
    return carpeta["id"]


def listar_nombres_archivos_en_carpeta(drive_id, carpeta_id, headers):
    """
    Devuelve un set con los nombres de los archivos existentes en esa carpeta.
    (Solo primera página; normalmente suficiente para este caso)
    """
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{carpeta_id}/children"
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print(f"⚠️ No se pudieron listar archivos de la carpeta salida: {resp.status_code}")
        return set()

    data = resp.json().get("value", [])
    return {it["name"] for it in data if "file" in it}


def subir_txt_a_onedrive(nombre_archivo, contenido, drive_id, carpeta_id, headers):
    """
    Sube un TXT a OneDrive dentro de la carpeta carpeta_id.
    Si existe, lo sobrescribe.
    """
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{carpeta_id}:/{nombre_archivo}:/content"
    resp = requests.put(
        url,
        headers={**headers, "Content-Type": "text/plain"},
        data=contenido.encode("utf-8")
    )
    if resp.status_code not in (200, 201):
        print(f"❌ Error subiendo {nombre_archivo}: {resp.status_code} - {resp.text}")
    else:
        print(f"✅ Subido a OneDrive: {nombre_archivo}")


# -------------------------------
# (2) Función para correr modelo
# -------------------------------

def correr_modelo(model_path, texto, instruccion):
    llm = Llama(
        model_path=model_path,
        n_ctx=PARAMS["n_ctx"],
        n_gpu_layers=N_GPU_LAYERS,
        n_batch=N_BATCH,
        n_threads=N_THREADS,
        use_mmap=True,
        verbose=False
    )
    prompt = f"### Instrucción:\n{instruccion}\n\n### TEXTO:\n{texto}\n\n### Respuesta:"
    resp = llm(prompt=prompt, max_tokens=PARAMS["max_tokens"], temperature=PARAMS["temperature"])
    salida = (resp["choices"][0]["text"] or "").strip()
    del llm
    gc.collect()
    return salida


# -------------------------------
# (3) Preparar carpeta de salida en OneDrive
# -------------------------------

NOMBRE_CARPETA_SALIDA = "orfeo_salida_llm"

# item_id es el ID del driveItem raíz del enlace compartido (ya lo calculaste antes)
carpeta_salida_id = obtener_o_crear_carpeta_salida(
    NOMBRE_CARPETA_SALIDA,
    drive_id,
    item_id,   # raíz del link de Orfeo
    headers
)

# Listar nombres existentes para poder hacer SKIP_IF_EXISTS
nombres_existentes = listar_nombres_archivos_en_carpeta(drive_id, carpeta_salida_id, headers)

# -------------------------------
# (4) Bucle principal usando df_pdfs
# -------------------------------

for idx, row in df_pdfs.iloc[:1].iterrows():
    archivo = str(row["nombre"])   # antes 'archivo'
    texto   = str(row["texto"])

    base_name = f"{archivo}__row{idx}"
    llama_name   = f"{base_name}__llama.txt"
    mistral_name = f"{base_name}__mistral.txt"

    # Si ya existen ambos en OneDrive, saltar
    if SKIP_IF_EXISTS and (llama_name in nombres_existentes) and (mistral_name in nombres_existentes):
        print(f"↪️ Ya existen ambos TXT para fila {idx} en OneDrive, se omite.")
        continue

    # Procesar cada modelo
    for modelo in MODELOS:
        nombre_modelo = modelo["name"]   # "llama" o "mistral"
        path_modelo   = modelo["path"]
        out_name = llama_name if nombre_modelo == "llama" else mistral_name

        if SKIP_IF_EXISTS and out_name in nombres_existentes:
            print(f"↪️ Ya existe {nombre_modelo} TXT para fila {idx} en OneDrive, se omite.")
            continue

        if not os.path.exists(path_modelo):
            print(f"⚠️ Modelo no encontrado: {path_modelo}")
            continue

        try:
            print(f"🧠 Procesando {nombre_modelo} en fila {idx}...")
            respuesta = correr_modelo(path_modelo, texto, INSTRUCCION)

            # Subir respuesta a OneDrive
            subir_txt_a_onedrive(out_name, respuesta, drive_id, carpeta_salida_id, headers)
            # Añadir al set local para evitar re-subir en esta misma corrida
            nombres_existentes.add(out_name)

        except Exception as e:
            print(f"❌ Error en modelo {nombre_modelo} para fila {idx}: {e}")

print(f"\n✅ Listo. Archivos TXT generados en la carpeta '{NOMBRE_CARPETA_SALIDA}' de OneDrive.")


📁 Carpeta 'orfeo_salida_llm' ya existe en OneDrive.
🧠 Procesando llama en fila 0...


llama_context: n_ctx_per_seq (2048) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


✅ Subido a OneDrive: 120253100125753.pdf__row0__llama.txt
🧠 Procesando mistral en fila 0...


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


✅ Subido a OneDrive: 120253100125753.pdf__row0__mistral.txt

✅ Listo. Archivos TXT generados en la carpeta 'orfeo_salida_llm' de OneDrive.


## 5. Consolidar y guardar los resultados

In [ ]:

import requests
import pandas as pd
import re

# ================================================================
# 1. LISTAR Y DESCARGAR ARCHIVOS TXT DESDE ONEDRIVE
# ================================================================

def listar_txt_en_carpeta(drive_id, carpeta_id, headers):
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{carpeta_id}/children"
    resp = requests.get(url, headers=headers)

    if resp.status_code != 200:
        raise Exception(f"Error listando TXT: {resp.status_code} - {resp.text}")

    data = resp.json().get("value", [])
    return [item for item in data if "file" in item and item["name"].endswith(".txt")]


def descargar_txt_onedrive(drive_id, item_id, headers):
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/content"
    resp = requests.get(url, headers=headers)

    if resp.status_code != 200:
        print(f"❌ Error descargando TXT {item_id}: {resp.status_code}")
        return ""

    # Garantizar UTF-8
    try:
        return resp.content.decode("utf-8", errors="replace").strip()
    except:
        return resp.text.strip()


# ================================================================
# 2. SEPARAR RESPUESTAS SEGÚN NUEVA INSTRUCCIÓN (4 RESPUESTAS)
# ================================================================

def separar_respuestas(texto):
    """
    Extrae:
    1) Tema principal
    2) Origen de la solicitud
    3) Tipo de solicitud
    4) Resumen en 7 palabras
    """

    tema = origen = tipo = resumen = ""

    r1 = re.search(r"1\).*?:\s*(.*)", texto)
    r2 = re.search(r"2\).*?:\s*(.*)", texto)
    r3 = re.search(r"3\).*?:\s*(.*)", texto)
    r4 = re.search(r"4\).*?:\s*(.*)", texto)

    if r1: tema = r1.group(1).strip()
    if r2: origen = r2.group(1).strip()
    if r3: tipo = r3.group(1).strip()
    if r4: resumen = r4.group(1).strip()

    return tema, origen, tipo, resumen


# ================================================================
# 3. LEER TODOS LOS TXT Y CONSTRUIR EL CONSOLIDADO
# ================================================================

print("📁 Listando archivos TXT en OneDrive...")
archivos_txt = listar_txt_en_carpeta(drive_id, carpeta_salida_id, headers)

filas = []

for item in archivos_txt:
    nombre = item["name"]
    file_id = item["id"]

    # Detectar modelo
    if "__llama" in nombre:
        base = nombre.replace("__llama.txt", "")
        modelo = "llama"
    else:
        base = nombre.replace("__mistral.txt", "")
        modelo = "mistral"

    # Descargar contenido
    contenido = descargar_txt_onedrive(drive_id, file_id, headers)

    # Separar respuestas
    tema, origen, tipo, resumen = separar_respuestas(contenido)

    # Buscar fila existente
    fila = next((x for x in filas if x["nombre_archivo"] == base), None)

    if not fila:
        fila = {
            "nombre_archivo": base,
            "llama": "",
            "mistral": "",
            "llama_tema": "",
            "llama_origen": "",
            "llama_tipo": "",
            "llama_resumen": "",
            "mistral_tema": "",
            "mistral_origen": "",
            "mistral_tipo": "",
            "mistral_resumen": ""
        }
        filas.append(fila)

    # Guardar contenido
    fila[modelo] = contenido
    fila[f"{modelo}_tema"] = tema
    fila[f"{modelo}_origen"] = origen
    fila[f"{modelo}_tipo"] = tipo
    fila[f"{modelo}_resumen"] = resumen


# ================================================================
# 4. CREAR DATAFRAMES
# ================================================================
df = pd.DataFrame(filas)

df_llama = df[[
    "nombre_archivo", "llama",
    "llama_tema", "llama_origen", "llama_tipo", "llama_resumen"
]]

df_mistral = df[[
    "nombre_archivo", "mistral",
    "mistral_tema", "mistral_origen", "mistral_tipo", "mistral_resumen"
]]


# ================================================================
# 5. GUARDAR LOS 3 EXCELS Y SUBIRLOS A ONEDRIVE (UTF-8)
# ================================================================



df.to_excel(EXCEL_CONSOLIDADO, index=False, engine="xlsxwriter")
df_llama.to_excel(EXCEL_LLAMA, index=False, engine="xlsxwriter")
df_mistral.to_excel(EXCEL_MISTRAL, index=False, engine="xlsxwriter")


def subir_a_onedrive(nombre_archivo):
    url_upload = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}:/{nombre_archivo}:/content"
    with open(nombre_archivo, "rb") as f:
        resp = requests.put(url_upload, headers=headers, data=f.read())

    if resp.status_code not in (200, 201):
        print(f"❌ Error subiendo {nombre_archivo}:", resp.status_code, resp.text)
    else:
        print(f"✅ Subido correctamente a OneDrive: {nombre_archivo}")


# Subir archivos
subir_a_onedrive(EXCEL_CONSOLIDADO)
subir_a_onedrive(EXCEL_LLAMA)
subir_a_onedrive(EXCEL_MISTRAL)

print("🎉 Todo listo: excels creados, separados, limpios y subidos a OneDrive.")


📁 Listando archivos TXT en OneDrive...
✅ Subido correctamente a OneDrive: Orfeo_analisis_llm.xlsx
✅ Subido correctamente a OneDrive: Orfeo_llama.xlsx
✅ Subido correctamente a OneDrive: Orfeo_mistral.xlsx
🎉 Todo listo: excels creados, separados, limpios y subidos a OneDrive.
